In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from integral_timber_joints.planning import load_pddlstream

Using pddlstream from C:\Users\harry\Documents\code_ws\pb_ws\coop_assembly\external\pddlstream\pddlstream\__init__.py


## Parse ITJ process

In [3]:
import os
from termcolor import cprint
import pybullet_planning as pp
from integral_timber_joints.planning.parsing import parse_process, save_process_and_movements, \
    get_process_path, save_process

In [4]:
# parse options
design_dir = '210916_SymbolicPlanning' # '210419_AnticlasticShelter' # '210605_ScrewdriverTestProcess' # 210419_AnticlasticShelter
problem = 'nine_pieces_process.json' #'shelter_process.json' # 'shelter_process.json' pavilion_process.json' # 'twelve_pieces_process.json'
problem_subdir = '.'

In [5]:
process = parse_process(design_dir, problem, subdir=problem_subdir)

# Double check entire solution is valid
for beam_id in process.assembly.sequence:
    if not process.dependency.beam_all_valid(beam_id):
        process.dependency.compute_all(beam_id)
        assert process.dependency.beam_all_valid(beam_id)

Process json parsed from c:\users\harry\dropbox (mit)\code_ws_dropbox\itj_ws\integral_timber_joints\external\itj_design_study\210916_SymbolicPlanning\nine_pieces_process.json


## Construct PDDL problem

In [6]:
from integral_timber_joints.planning.pddlstream_definitions.problem import get_pddlstream_problem

In [7]:
debug = 0
verbose = 1
use_partial_order = 1
reset_to_home = 0

In [8]:
pddlstream_problem = get_pddlstream_problem(process, use_partial_order=use_partial_order, debug=debug, 
                                            reset_to_home=reset_to_home)

In [9]:
print('Init:', pddlstream_problem.init)
print()
print('Goal:', pddlstream_problem.goal)

Init: [('RobotConf', Conf-0.93), ('RobotAtConf', Conf-0.93), ('CanMove',), ('RobotToolChangerEmpty',), ('Element', 'b0'), ('AtRack', 'b0'), ('IsElement', 'b0'), ('Grounded', 'b0'), ('Element', 'b1'), ('AtRack', 'b1'), ('IsElement', 'b1'), ('Grounded', 'b1'), ('Element', 'b2'), ('AtRack', 'b2'), ('IsElement', 'b2'), ('Element', 'b3'), ('AtRack', 'b3'), ('IsElement', 'b3'), ('Element', 'b5'), ('AtRack', 'b5'), ('IsElement', 'b5'), ('Grounded', 'b5'), ('Element', 'b6'), ('AtRack', 'b6'), ('IsElement', 'b6'), ('Element', 'b7'), ('AtRack', 'b7'), ('IsElement', 'b7'), ('Grounded', 'b7'), ('Element', 'b8'), ('AtRack', 'b8'), ('IsElement', 'b8'), ('Element', 'b4'), ('AtRack', 'b4'), ('IsElement', 'b4'), ('Joint', 'b0', 'b2'), ('NoToolAtJoint', 'b0', 'b2'), ('Joint', 'b0', 'b3'), ('NoToolAtJoint', 'b0', 'b3'), ('Joint', 'b0', 'b6'), ('NoToolAtJoint', 'b0', 'b6'), ('Joint', 'b1', 'b2'), ('NoToolAtJoint', 'b1', 'b2'), ('Joint', 'b1', 'b3'), ('NoToolAtJoint', 'b1', 'b3'), ('Joint', 'b1', 'b8'), ('

## Solve

In [34]:
from pddlstream.algorithms.downward import set_cost_scale, parse_action
from pddlstream.algorithms.meta import solve
from pddlstream.utils import INF
from pddlstream.language.constants import print_plan, is_plan

from integral_timber_joints.planning.pddlstream_definitions.problem import get_pddlstream_problem

pddlstream_problem = get_pddlstream_problem(process, use_partial_order=True, debug=True,
                                            reset_to_home=True)

# print('Init:', pddlstream_problem.init)
print()
print('Goal:', pddlstream_problem.goal)
print()

costs = True
set_cost_scale(1)

solution = solve(pddlstream_problem, algorithm='incremental', 
                 max_time=60,
                 unit_costs=True,
                 max_planner_time=300, 
                 debug=0, verbose=0)

plan, cost, evaluations = solution
plan_success = is_plan(plan)
cprint('Planning {}'.format('succeeds' if plan_success else 'fails'), 'green' if plan_success else 'red')

print('-'*10)
print_plan(plan)


Goal: ('and', ('Assembled', 'b0'), ('Assembled', 'b1'), ('Assembled', 'b2'), ('Assembled', 'b3'), ('Assembled', 'b5'), ('Assembled', 'b6'), ('Assembled', 'b7'), ('Assembled', 'b8'), ('Assembled', 'b4'), ('AtRack', 'g1'), ('AtRack', 'g2'), ('AtRack', 'g3'))

Iteration: 1 | Complexity: 0 | Calls: 0 | Evaluations: 95 | Solved: False | Cost: inf | Search Time: 0.000 | Sample Time: 0.000 | Time: 0.000
Iteration: 2 | Complexity: 1 | Calls: 24 | Evaluations: 191 | Solved: False | Cost: inf | Search Time: 0.031 | Sample Time: 0.000 | Time: 0.031
Summary: {complexity: 1, cost: 20.000, evaluations: 191, iterations: 2, length: 20, run_time: 0.125, sample_time: 0.000, search_time: 0.125, solutions: 1, solved: True, timeout: False}
Planning succeeds
----------
 1) pick_tool_from_rack g1 @conf1285 @conf2285 @traj285
 2) pick_element_from_rack b0 @conf1267 @conf2267 @traj267 g1
 3) place_element_on_structure b0 @conf1268 @conf2268 @traj268 g1
 4) pick_element_from_rack b1 @conf1269 @conf2269 @traj26

In [25]:
# from pddlstream.algorithms.algorithm import parse_problem
# from pddlstream.algorithms.constraints import PlanConstraints
# from pddlstream.algorithms.downward import get_problem, task_from_domain_problem
# from pddlstream.algorithms.meta import examine_instantiated

# pddlstream_problem = get_pddlstream_problem(process, use_partial_order=True, debug=False,
#                                             reset_to_home=False)

# print(pddlstream_problem.goal)

# results, instantiated = examine_instantiated(pddlstream_problem, unit_costs=1, verbose=0, debug=1)

In [28]:
plan, cost, evaluations = solution
# print('-'*10)
print_plan(plan)

# actions = [action for action in plan if action.name == 'add_element' or action.name == 'add_scaffold_element']
# element_sequence = [ac.args[2] for ac in actions]

In [30]:
evaluations.all_facts

[('robotconf', Conf-0.89),
 ('robotatconf', Conf-0.89),
 ('canmove',),
 ('robottoolchangerempty',),
 ('element', 'b0'),
 ('atrack', 'b0'),
 ('iselement', 'b0'),
 ('grounded', 'b0'),
 ('element', 'b1'),
 ('atrack', 'b1'),
 ('iselement', 'b1'),
 ('grounded', 'b1'),
 ('element', 'b2'),
 ('atrack', 'b2'),
 ('iselement', 'b2'),
 ('element', 'b3'),
 ('atrack', 'b3'),
 ('iselement', 'b3'),
 ('element', 'b5'),
 ('atrack', 'b5'),
 ('iselement', 'b5'),
 ('grounded', 'b5'),
 ('element', 'b6'),
 ('atrack', 'b6'),
 ('iselement', 'b6'),
 ('element', 'b7'),
 ('atrack', 'b7'),
 ('iselement', 'b7'),
 ('grounded', 'b7'),
 ('element', 'b8'),
 ('atrack', 'b8'),
 ('iselement', 'b8'),
 ('element', 'b4'),
 ('atrack', 'b4'),
 ('iselement', 'b4'),
 ('joint', 'b0', 'b2'),
 ('notoolatjoint', 'b0', 'b2'),
 ('joint', 'b0', 'b3'),
 ('notoolatjoint', 'b0', 'b3'),
 ('joint', 'b0', 'b6'),
 ('notoolatjoint', 'b0', 'b6'),
 ('joint', 'b1', 'b2'),
 ('notoolatjoint', 'b1', 'b2'),
 ('joint', 'b1', 'b3'),
 ('notoolatjoint', 

In [25]:
process.assembly.sequence

['b0', 'b1', 'b2', 'b3', 'b5', 'b6', 'b7', 'b8', 'b4']

In [ ]:
# TODO visualize sequence in a pybullet window